Let us agggregate data in 5-day intervals. That way we can use the variables whose temporal resolution are of 5-day granularity and also shortens the length of time series. It is likely(?) that we do not need track every thing on a daily basis.

- Dec. 29, 2025
- HN

In [56]:
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import pandas as pd
import numpy as np
import random
import os, os.path, pickle, sys

import matplotlib
import matplotlib.pyplot as plt

from typing import List, Optional, Any

import configparser
import argparse

# from pathlib import Path
# PROJECT_ROOT = Path.cwd().parent
# sys.path.insert(0, str(PROJECT_ROOT))
# from transformersCores import config
# from transformersCores import config as cfg
import sys
sys.path.append('./../')
import transformersCores.config as cfg
import transformersCores.preprocess as prp

In [ ]:
cfg

<module 'transformersCores.config' from 'd:\\github\\wheat_transformers\\notebooks_drivers\\01_pre-process\\..\\transformersCores\\config.py'>

: 

In [4]:
import importlib
importlib.reload(cfg);
importlib.reload(prp);

In [5]:
#dir_base = "./../../../01_research_data/Ehsan/wheat/"
dir_base = "D:/01_research_data/Wheat/Data/"
#data_base = dir_base + "Data/"
data_base = "D:/01_research_data/Wheat/Data/"
raw_dir = os.path.join(data_base, "00_raw_by_location/")
configs_dir = dir_base + "configs/"

In [ ]:
params = cfg.aggregateParameters()
params.readFromFile(configs_dir + "Scenario_1_aggr_cols.cfg")
# params = vars(params)

In [59]:
params

{'avg_rh_perc': 'mean',
 'avg_temp_c': 'mean',
 'cum_gdd_cday': 'sum',
 'dailyGDD_diff_cday': 'mean',
 'dap': 'mean',
 'diurnal_temp_range_c': 'mean',
 'freezing_dd_cday': 'mean',
 'heat_dd_cday': 'mean',
 'max_rh_perc': 'mean',
 'max_temp_c': 'mean',
 'min_rh_perc': 'mean',
 'min_temp_c': 'mean',
 'potential_evapo_mmday': 'mean',
 'precip_dtr_mmdayc': 'mean',
 'precip_mmday': 'sum',
 'specific_humidity_kgkg': 'mean',
 'sr_wm2': 'mean',
 'vpd_kpa': 'mean',
 'wet_day_frequency_days': 'mean',
 'wind_speed_ms': 'mean',
 'Author': 'HN',
 'Notebook': 'create_aggregation_config_files',
 'Note': 'Aggregate daily data into 5-day resolution. Scenario 1',
 'Date': '2026-01-06 11:23:08'}

In [7]:
# params_v2 = cfg.aggregateParameters()
# params_v2.load_config_from_file(configs_dir + "Scenario_1_aggr_cols.cfg")

### Read Data

In [8]:
filename = raw_dir + "raw_all_locations_separate_daily_monthly_fiveDay.sav"
data_dict = pd.read_pickle(filename)

In [9]:
data_dict.keys()

dict_keys(['wheat_all_locs_raw_daily', 'wheat_all_locs_raw_monthly', 'wheat_all_locs_raw_fiveDay', 'varieties_traits', 'source_code', 'Author', 'Date', 'Note'])

In [10]:
raw_daily = data_dict["wheat_all_locs_raw_daily"]
raw_fiveDay = data_dict["wheat_all_locs_raw_fiveDay"]
varieties_traits = data_dict["varieties_traits"]

In [11]:
raw_daily.head(2)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday,year,planting_date,harvest_date
0,almira,2001-01-01,1,0,Emergence,0.0,0.15,-7.45,0.03,0.0,...,0.0,7.6,0.0,0.0,0,11.68,0.0,2001,NaN,NaN
1,almira,2001-01-02,2,1,Emergence,0.0,0.75,-8.05,0.04,0.2,...,0.0,8.8,0.0,0.0,0,12.28,0.0,2001,NaN,NaN


In [12]:
params = cfg.aggregateParameters()
params.readFromFile(configs_dir + "Scenario_1_aggr_cols.cfg")

daily_converted_2_5day = prp.aggregate_per5day(params=params, daily_df=raw_daily, fiveDay_df=raw_fiveDay)

In [13]:
raw_daily[(raw_daily["location"] == "walla_walla") & (raw_daily["year"] == 2003)].head(5)

,location,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,...,dailyGDD_diff_cday,diurnal_temp_range_c,precip_dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday,year,planting_date,harvest_date
261130,walla_walla,2003-01-01,1,0,Emergence,2.2,7.35,0.55,0.19,0.9,...,0.75,6.8,0.323529,3.95,1,3.68,0.0,2003,2003-03-24,2003-07-25
261131,walla_walla,2003-01-02,2,1,Emergence,6.1,11.75,5.45,0.29,1.6,...,4.65,6.3,0.968254,12.55,1,0.00,0.0,2003,2003-03-24,2003-07-25
261132,walla_walla,2003-01-03,3,2,Emergence,2.4,10.95,6.85,0.31,1.3,...,0.30,4.1,0.585366,21.45,1,0.00,0.0,2003,2003-03-24,2003-07-25
261133,walla_walla,2003-01-04,4,3,Emergence,3.7,10.55,7.15,0.27,1.4,...,0.05,3.4,1.088235,30.30,1,0.00,0.0,2003,2003-03-24,2003-07-25
261134,walla_walla,2003-01-05,5,4,Emergence,0.0,8.35,2.75,0.24,0.9,...,3.30,5.6,0.000000,35.85,0,1.48,0.0,2003,2003-03-24,2003-07-25


In [14]:
raw_fiveDay[(raw_fiveDay["date"].dt.year == 2003) & (raw_fiveDay["location"] == "walla_walla")].head(5)

,date,doy,location,std_precip_evap_unitless,palmer_drought_unitless
52195,2003-01-05,5,walla_walla,0.90,-3.53
52196,2003-01-10,10,walla_walla,0.90,-3.45
52197,2003-01-15,15,walla_walla,-0.17,-3.40
52198,2003-01-20,20,walla_walla,-0.80,-3.37
52199,2003-01-25,25,walla_walla,0.03,-3.30


In [15]:
end_date = pd.to_datetime("2003-01-05")
loc = "walla_walla"
# end_date = raw_fiveDay["date"][0]
five_days = pd.date_range(end=end_date, periods=5, freq='D')
a_col = "precip_mmday"

raw_daily[(raw_daily["location"] == loc) & (raw_daily["date"].isin(five_days))][a_col].sum()

np.float64(14.400000000000002)

In [16]:
a_col = "sum_precip_mmday"
daily_converted_2_5day[(daily_converted_2_5day["location"] == loc) & \
                       (daily_converted_2_5day["date"] == end_date)][a_col]

52195    14.4
Name: sum_precip_mmday, dtype: float64

In [17]:
daily_converted_2_5day.head(2)

,location,date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,mean_heat_dd_cday,...,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,stage,planting_date,harvest_date
0,almira,2001-01-05,89.48,-1.75,0.95,0.19,2.0,6.48,9.22,0.0,...,0.0,0.0,0.00337,48.92,0.048,0.0,2.18,Emergence,NaN,NaN
1,almira,2001-01-10,85.62,-1.49,4.75,0.19,7.0,7.16,9.30,0.0,...,0.0,0.0,0.00312,59.60,0.098,0.0,2.38,Emergence,NaN,NaN


In [18]:
raw_daily[(raw_daily["location"] == "walla_walla") & (raw_daily["year"] == 2001)]["planting_date"].unique()

array([nan], dtype=object)

In [19]:
daily_converted_2_5day['date'] = pd.to_datetime(daily_converted_2_5day['date'])
daily_converted_2_5day['planting_date'] = pd.to_datetime(daily_converted_2_5day['planting_date'])
daily_converted_2_5day['harvest_date'] = pd.to_datetime(daily_converted_2_5day['harvest_date'])

In [20]:
daily_converted_2_5day_backup = daily_converted_2_5day.copy()

In [21]:
daily_converted_2_5day = daily_converted_2_5day_backup.copy()

In [22]:
# daily_converted_2_5day.dropna(subset=['planting_date'], inplace=True)
# daily_converted_2_5day.sort_values(["location", "date"], inplace=True)
# daily_converted_2_5day.reset_index(drop=True, inplace=True)

# daily_converted_2_5day.head(3)

In [23]:
daily_converted_2_5day_backup.head(3)

,location,date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,mean_heat_dd_cday,...,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,stage,planting_date,harvest_date
0,almira,2001-01-05,89.48,-1.75,0.95,0.19,2.0,6.48,9.22,0.0,...,0.000000,0.0,0.003370,48.92,0.048,0.0,2.18,Emergence,NaT,NaT
1,almira,2001-01-10,85.62,-1.49,4.75,0.19,7.0,7.16,9.30,0.0,...,0.000000,0.0,0.003120,59.60,0.098,0.0,2.38,Emergence,NaT,NaT
2,almira,2001-01-15,88.86,-1.57,4.75,0.00,12.0,6.36,8.98,0.0,...,0.021336,0.6,0.003366,46.32,0.056,0.4,2.06,Emergence,NaT,NaT


In [24]:
super_df_oct = prp.TS_df_givenMonth_prior2Plant(daily_converted_2_5day_backup, start_month=10)

## Check point

In [25]:
super_df_oct.head(3)

,location,planting_date,harvest_date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,...,mean_min_rh_perc,mean_min_temp_c,mean_potential_evapo_mmday,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms
0,almira,2002-04-08,2002-08-12,date 2001-10-02 50.22 2001-10-07 41.92 2...,date 2001-10-02 13.88 2001-10-07 9.79 2...,date 2001-10-02 15200.40 2001-10-07 1550...,date 2001-10-02 1.55 2001-10-07 2.13 200...,date 2001-10-02 272.0 2001-10-07 277.0 2...,date 2001-10-02 15.18 2001-10-07 12.96 2...,date 2001-10-02 8.16 2001-10-07 11.14 2...,...,date 2001-10-02 26.10 2001-10-07 21.52 2...,date 2001-10-02 6.29 2001-10-07 3.31 2...,date 2001-10-02 3.04 2001-10-07 2.78 200...,date 2001-10-02 0.000000 2001-10-07 0.00...,date 2001-10-02 0.0 2001-10-07 0.0 2001-...,date 2001-10-02 0.004296 2001-10-07 0.00...,date 2001-10-02 169.10 2001-10-07 148.38...,date 2001-10-02 1.130 2001-10-07 0.844 2...,date 2001-10-02 0.0 2001-10-07 0.0 2001-...,date 2001-10-02 2.58 2001-10-07 3.26 200...
1,almira,2003-04-29,2003-08-20,date 2002-10-02 58.14 2002-10-07 66.29 2...,date 2002-10-02 9.59 2002-10-07 11.47 2...,date 2002-10-02 14826.90 2002-10-07 1506...,date 2002-10-02 1.30 2002-10-07 1.38 200...,date 2002-10-02 272.0 2002-10-07 277.0 2...,date 2002-10-02 12.72 2002-10-07 12.92 2...,date 2002-10-02 11.22 2002-10-07 9.44 2...,...,date 2002-10-02 32.16 2002-10-07 41.08 2...,date 2002-10-02 3.23 2002-10-07 5.01 2...,date 2002-10-02 2.52 2002-10-07 2.22 200...,date 2002-10-02 0.033273 2002-10-07 0.05...,date 2002-10-02 1.7 2002-10-07 2.1 2002-...,date 2002-10-02 0.004288 2002-10-07 0.00...,date 2002-10-02 139.42 2002-10-07 130.20...,date 2002-10-02 0.666 2002-10-07 0.604 2...,date 2002-10-02 0.4 2002-10-07 0.4 2002-...,date 2002-10-02 3.46 2002-10-07 3.02 200...
2,almira,2004-04-01,2004-08-12,date 2003-10-02 41.24 2003-10-07 36.67 2...,date 2003-10-02 17.57 2003-10-07 17.18 2...,date 2003-10-02 16545.05 2003-10-07 1698...,date 2003-10-02 1.25 2003-10-07 1.21 200...,date 2003-10-02 272.0 2003-10-07 277.0 2...,date 2003-10-02 14.76 2003-10-07 12.50 2...,date 2003-10-02 4.26 2003-10-07 3.52 2...,...,date 2003-10-02 22.62 2003-10-07 20.18 2...,date 2003-10-02 10.19 2003-10-07 10.93 2...,date 2003-10-02 4.14 2003-10-07 3.26 200...,date 2003-10-02 0.000000 2003-10-07 0.00...,date 2003-10-02 0.0 2003-10-07 0.0 200...,date 2003-10-02 0.004892 2003-10-07 0.00...,date 2003-10-02 174.36 2003-10-07 143.36...,date 2003-10-02 1.478 2003-10-07 1.462 2...,date 2003-10-02 0.0 2003-10-07 0.0 2003-...,date 2003-10-02 3.66 2003-10-07 3.00 200...


In [26]:
t2 = super_df_oct.loc[0, "harvest_date"]
t1 = super_df_oct.loc[0, "planting_date"]
t1 = pd.Timestamp(f"{t1.year - 1}-10-01")

days_between = (t2 - t1).days
days_between

315

In [27]:
daily_converted_2_5day[(daily_converted_2_5day["location"] == "almira") & \
                       (daily_converted_2_5day['date'].dt.year == 2002)].head(3)

,location,date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,mean_heat_dd_cday,...,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,stage,planting_date,harvest_date
73,almira,2002-01-05,88.62,-1.67,0.00,0.00,2.0,6.76,9.28,0.0,...,0.270819,8.7,0.003320,45.36,0.056,0.4,2.1,Emergence,2002-04-08,2002-08-12
74,almira,2002-01-10,86.42,1.32,19.95,1.02,7.0,5.86,5.84,0.0,...,0.399623,10.2,0.004014,37.54,0.084,0.4,2.9,Emergence,2002-04-08,2002-08-12
75,almira,2002-01-15,82.62,-1.38,43.95,0.92,12.0,7.98,9.60,0.0,...,0.012308,0.4,0.003086,50.80,0.116,0.2,3.0,Emergence,2002-04-08,2002-08-12


In [28]:
locations_list = daily_converted_2_5day["location"].unique()
loc_ = locations_list[0]

loc_yr = daily_converted_2_5day.loc[daily_converted_2_5day["location"] == \
                                      loc_, "harvest_date"].unique()[-1]
loc_yr

Timestamp('2023-08-21 00:00:00')

In [29]:
test_super = super_df_oct[(super_df_oct["location"] == loc_) & (super_df_oct["harvest_date"] == loc_yr)]
test_super

,location,planting_date,harvest_date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,...,mean_min_rh_perc,mean_min_temp_c,mean_potential_evapo_mmday,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms
20,almira,2023-04-26,2023-08-21,date 2022-10-02 59.32 2022-10-07 45.60 2...,date 2022-10-02 16.33 2022-10-07 18.64 2...,date 2022-10-02 14853.65 2022-10-07 1530...,date 2022-10-02 2.77 2022-10-07 0.55 202...,date 2022-10-02 272.0 2022-10-07 277.0 2...,date 2022-10-02 11.92 2022-10-07 13.70 2...,date 2022-10-02 4.08 2022-10-07 2.66 202...,...,date 2022-10-02 35.88 2022-10-07 22.30 2...,date 2022-10-02 10.37 2022-10-07 11.79 2...,date 2022-10-02 2.70 2022-10-07 3.00 202...,date 2022-10-02 0.022388 2022-10-07 0.00...,date 2022-10-02 1.5 2022-10-07 0.0 202...,date 2022-10-02 0.006816 2022-10-07 0.00...,date 2022-10-02 155.38 2022-10-07 158.18...,date 2022-10-02 0.982 2022-10-07 1.400 2...,date 2022-10-02 0.2 2022-10-07 0.0 2022-...,date 2022-10-02 2.30 2022-10-07 2.24 202...


In [30]:
start_date = pd.Timestamp(year=loc_yr.year - 1, month=10, day=1)

test_df = daily_converted_2_5day[(daily_converted_2_5day["location"] == loc_) & \
                                 (daily_converted_2_5day["date"] >= start_date) & \
                                 (daily_converted_2_5day["date"] <= loc_yr)]

In [31]:
a_col = "mean_dailyGDD_diff_cday"
sum(test_df[a_col].values - test_super[a_col].item().values)

np.float64(0.0)

### Time series 3 months prior planting date as opposed to prior Oct.

In [32]:
import importlib
importlib.reload(cfg);
importlib.reload(prp);

In [33]:
super_df_3months = prp.TS_df_countMonths_prior2Plant(daily_converted_2_5day_backup, months_prior=3)

## Check point

In [34]:
locations_list = daily_converted_2_5day["location"].unique()
loc_ = locations_list[0]

loc_yr = daily_converted_2_5day.loc[daily_converted_2_5day["location"] == \
                                      loc_, "planting_date"].unique()[2]
loc_yr

Timestamp('2003-04-29 00:00:00')

In [35]:
test_super = super_df_3months[(super_df_3months["location"] == loc_) & \
                              (super_df_3months["planting_date"] == loc_yr)]
test_super

,location,planting_date,harvest_date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,...,mean_min_rh_perc,mean_min_temp_c,mean_potential_evapo_mmday,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms
1,almira,2003-04-29,2003-08-20,date 2003-01-05 87.44 2003-01-10 81.31 2...,date 2003-01-05 2.30 2003-01-10 -1.93 2...,date 2003-01-05 29.05 2003-01-10 5...,date 2003-01-05 9.800000e-01 2003-01-10 ...,date 2003-01-05 2.0 2003-01-10 7.0 2...,date 2003-01-05 4.38 2003-01-10 7.08 2...,date 2003-01-05 4.120 2003-01-10 9.700...,...,date 2003-01-05 74.88 2003-01-10 65.96 2...,date 2003-01-05 0.11 2003-01-10 -5.47 2...,date 2003-01-05 0.30 2003-01-10 0.32 200...,date 2003-01-05 1.371110 2003-01-10 0.00...,date 2003-01-05 24.2 2003-01-10 0.0 200...,date 2003-01-05 0.004370 2003-01-10 0.00...,date 2003-01-05 40.90 2003-01-10 61.34...,date 2003-01-05 0.078 2003-01-10 0.114 2...,date 2003-01-05 0.8 2003-01-10 0.0 2003-...,date 2003-01-05 3.26 2003-01-10 2.56 200...


In [36]:
super_df_3months

,location,planting_date,harvest_date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,...,mean_min_rh_perc,mean_min_temp_c,mean_potential_evapo_mmday,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms
0,almira,2002-04-08,2002-08-12,date 2002-01-05 88.62 2002-01-10 86.42 2...,date 2002-01-05 -1.67 2002-01-10 1.32 2...,date 2002-01-05 0.00 2002-01-10 19....,date 2002-01-05 0.00 2002-01-10 1.02 200...,date 2002-01-05 2.0 2002-01-10 7.0 2...,date 2002-01-05 6.76 2002-01-10 5.86 2...,date 2002-01-05 9.280 2002-01-10 5.840...,...,date 2002-01-05 77.24 2002-01-10 72.84 2...,date 2002-01-05 -5.05 2002-01-10 -1.61 2...,date 2002-01-05 0.20 2002-01-10 0.32 200...,date 2002-01-05 0.270819 2002-01-10 0.39...,date 2002-01-05 8.7 2002-01-10 10.2 200...,date 2002-01-05 0.003320 2002-01-10 0.00...,date 2002-01-05 45.36 2002-01-10 37.54...,date 2002-01-05 0.056 2002-01-10 0.084 2...,date 2002-01-05 0.4 2002-01-10 0.4 2002-...,date 2002-01-05 2.10 2002-01-10 2.90 200...
1,almira,2003-04-29,2003-08-20,date 2003-01-05 87.44 2003-01-10 81.31 2...,date 2003-01-05 2.30 2003-01-10 -1.93 2...,date 2003-01-05 29.05 2003-01-10 5...,date 2003-01-05 9.800000e-01 2003-01-10 ...,date 2003-01-05 2.0 2003-01-10 7.0 2...,date 2003-01-05 4.38 2003-01-10 7.08 2...,date 2003-01-05 4.120 2003-01-10 9.700...,...,date 2003-01-05 74.88 2003-01-10 65.96 2...,date 2003-01-05 0.11 2003-01-10 -5.47 2...,date 2003-01-05 0.30 2003-01-10 0.32 200...,date 2003-01-05 1.371110 2003-01-10 0.00...,date 2003-01-05 24.2 2003-01-10 0.0 200...,date 2003-01-05 0.004370 2003-01-10 0.00...,date 2003-01-05 40.90 2003-01-10 61.34...,date 2003-01-05 0.078 2003-01-10 0.114 2...,date 2003-01-05 0.8 2003-01-10 0.0 2003-...,date 2003-01-05 3.26 2003-01-10 2.56 200...
2,almira,2004-04-01,2004-08-12,date 2004-01-05 73.26 2004-01-10 82.96 2...,date 2004-01-05 -9.59 2004-01-10 -5.61 2...,date 2004-01-05 0.00 2004-01-10 ...,date 2004-01-05 0.00 2004-01-10 0.00 200...,date 2004-01-05 2.0 2004-01-10 7.0 2...,date 2004-01-05 6.08 2004-01-10 6.32 2...,date 2004-01-05 16.860 2004-01-10 13.000...,...,date 2004-01-05 63.72 2004-01-10 75.98 2...,date 2004-01-05 -12.63 2004-01-10 -8.77 2...,date 2004-01-05 0.34 2004-01-10 0.26 200...,date 2004-01-05 0.100909 2004-01-10 0.48...,date 2004-01-05 2.1 2004-01-10 16.9 200...,date 2004-01-05 0.001704 2004-01-10 0.00...,date 2004-01-05 47.60 2004-01-10 42.08...,date 2004-01-05 0.078 2004-01-10 0.074 2...,date 2004-01-05 0.4 2004-01-10 1.0 2004-...,date 2004-01-05 3.32 2004-01-10 2.16 200...
3,almira,2005-04-14,2005-08-16,date 2005-01-05 81.133333 2005-01-10 86....,date 2005-01-05 -6.233333 2005-01-10 -6....,date 2005-01-05 3763.60 2005-01-10 ...,date 2005-01-05 0.00 2005-01-10 0.00 200...,date 2005-01-05 62.5 2005-01-10 7.0 2...,date 2005-01-05 4.70 2005-01-10 5.86 2...,date 2005-01-05 14.516667 2005-01-10 13....,...,date 2005-01-05 71.683333 2005-01-10 77....,date 2005-01-05 -8.583333 2005-01-10 -8....,date 2005-01-05 0.233333 2005-01-10 0.26...,date 2005-01-05 0.087302 2005-01-10 0.40...,date 2005-01-05 1.1 2005-01-10 10.6 200...,date 2005-01-05 0.002270 2005-01-10 0.00...,date 2005-01-05 48.40 2005-01-10 33.18...,date 2005-01-05 0.061667 2005-01-10 0.05...,date 2005-01-05 0.166667 2005-01-10 0.80...,date 2005-01-05 3.316667 2005-01-10 3.06...
4,almira,2006-04-18,2006-08-10,date 2006-01-05 88.71 2006-01-10 87.94 2...,date 2006-01-05 1.14 2006-01-10 3.10 2...,date 2006-01-05 16.00 2006-01-10 7...,date 2006-01-05 0.50 2006-01-10 1.64 200...,date 2006-01-05 2.0 2006-01-10 7.0 2...,date 2006-01-05 5.38 2006-01-10 5.14 2...,date 2006-01-05 5.780 2006-01-10 3.700...,...,date 2006-01-05 77.42 2006-01-10 75.88 2...,date 2006-01-05 -1.55 2006-01-10 0.53 2...,date 2006-01-05 0.20 2006-01-10 0.38 200...,date 2006-01-05 0.545599 2006-01-10 1.08...,date 2006-01-05 13.8 2006-01-10 29.5 200...,date 2006-01-05

In [37]:
three_months_prior = loc_yr - pd.DateOffset(months=3)
start_date = three_months_prior.replace(day=1)

harvest_date = daily_converted_2_5day.loc[(daily_converted_2_5day["location"] == loc_) & \
                           (daily_converted_2_5day["planting_date"] == loc_yr)]["harvest_date"].unique()[0]

test_df = daily_converted_2_5day[(daily_converted_2_5day["location"] == loc_) & \
                                 (daily_converted_2_5day["date"] >= start_date) & \
                                 (daily_converted_2_5day["date"] <= harvest_date)]
test_df.head(3)

,location,date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,mean_heat_dd_cday,...,mean_precip_dtr_mmdayc,sum_precip_mmday,mean_specific_humidity_kgkg,mean_sr_wm2,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,stage,planting_date,harvest_date
146,almira,2003-01-05,87.44,2.30,29.05,0.98,2.0,4.38,4.12,0.0,...,1.371110,24.2,0.004370,40.90,0.078,0.8,3.26,Emergence,2003-04-29,2003-08-20
147,almira,2003-01-10,81.31,-1.93,58.75,0.48,7.0,7.08,9.70,0.0,...,0.000000,0.0,0.002918,61.34,0.114,0.0,2.56,Emergence,2003-04-29,2003-08-20
148,almira,2003-01-15,86.49,-0.04,62.10,0.52,12.0,4.34,6.44,0.0,...,1.332444,19.1,0.003696,45.84,0.062,0.4,1.76,Emergence,2003-04-29,2003-08-20


In [38]:
a_col = "mean_dap"
sum(test_df[a_col].values - test_super[a_col].item().values)

np.float64(0.0)

In [39]:
#### Read variety performance traits
varieties_traits = data_dict["varieties_traits"] 

In [40]:
cols_to_use = [
    "location",
    "year",
    "variety",
    "grain_yield",
    "plant_height",
    "heading_date",
    "protein_content",
    "test_weight"
]

In [41]:
super_df_3months['year'] = super_df_3months['planting_date'].dt.year


In [42]:
merged_super_trait = super_df_3months.merge(varieties_traits[cols_to_use], on=['location', 'year'], how="left")


In [43]:
merged_super_trait[merged_super_trait['variety'] == 'Alpowa']

,location,planting_date,harvest_date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,...,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,year,variety,grain_yield,plant_height,heading_date,protein_content,test_weight
0,almira,2002-04-08,2002-08-12,date 2002-01-05 88.62 2002-01-10 86.42 2...,date 2002-01-05 -1.67 2002-01-10 1.32 2...,date 2002-01-05 0.00 2002-01-10 19....,date 2002-01-05 0.00 2002-01-10 1.02 200...,date 2002-01-05 2.0 2002-01-10 7.0 2...,date 2002-01-05 6.76 2002-01-10 5.86 2...,date 2002-01-05 9.280 2002-01-10 5.840...,...,date 2002-01-05 0.056 2002-01-10 0.084 2...,date 2002-01-05 0.4 2002-01-10 0.4 2002-...,date 2002-01-05 2.10 2002-01-10 2.90 200...,2002,Alpowa,25.666667,25.666667,NaN,11.300000,58.866667
46,almira,2003-04-29,2003-08-20,date 2003-01-05 87.44 2003-01-10 81.31 2...,date 2003-01-05 2.30 2003-01-10 -1.93 2...,date 2003-01-05 29.05 2003-01-10 5...,date 2003-01-05 9.800000e-01 2003-01-10 ...,date 2003-01-05 2.0 2003-01-10 7.0 2...,date 2003-01-05 4.38 2003-01-10 7.08 2...,date 2003-01-05 4.120 2003-01-10 9.700...,...,date 2003-01-05 0.078 2003-01-10 0.114 2...,date 2003-01-05 0.8 2003-01-10 0.0 2003-...,date 2003-01-05 3.26 2003-01-10 2.56 200...,2003,Alpowa,29.000000,29.000000,182.666667,10.500000,59.000000
87,almira,2004-04-01,2004-08-12,date 2004-01-05 73.26 2004-01-10 82.96 2...,date 2004-01-05 -9.59 2004-01-10 -5.61 2...,date 2004-01-05 0.00 2004-01-10 ...,date 2004-01-05 0.00 2004-01-10 0.00 200...,date 2004-01-05 2.0 2004-01-10 7.0 2...,date 2004-01-05 6.08 2004-01-10 6.32 2...,date 2004-01-05 16.860 2004-01-10 13.000...,...,date 2004-01-05 0.078 2004-01-10 0.074 2...,date 2004-01-05 0.4 2004-01-10 1.0 2004-...,date 2004-01-05 3.32 2004-01-10 2.16 200...,2004,Alpowa,29.333333,29.333333,171.333333,13.400000,54.933333
123,almira,2005-04-14,2005-08-16,date 2005-01-05 81.133333 2005-01-10 86....,date 2005-01-05 -6.233333 2005-01-10 -6....,date 2005-01-05 3763.60 2005-01-10 ...,date 2005-01-05 0.00 2005-01-10 0.00 200...,date 2005-01-05 62.5 2005-01-10 7.0 2...,date 2005-01-05 4.70 2005-01-10 5.86 2...,date 2005-01-05 14.516667 2005-01-10 13....,...,date 2005-01-05 0.061667 2005-01-10 0.05...,date 2005-01-05 0.166667 2005-01-10 0.80...,date 2005-01-05 3.316667 2005-01-10 3.06...,2005,Alpowa,32.000000,32.000000,175.666667,9.366667,58.966667
165,almira,2006-04-18,2006-08-10,date 2006-01-05 88.71 2006-01-10 87.94 2...,date 2006-01-05 1.14 2006-01-10 3.10 2...,date 2006-01-05 16.00 2006-01-10 7...,date 2006-01-05 0.50 2006-01-10 1.64 200...,date 2006-01-05 2.0 2006-01-10 7.0 2...,date 2006-01-05 5.38 2006-01-10 5.14 2...,date 2006-01-05 5.780 2006-01-10 3.700...,...,date 2006-01-05 0.038 2006-01-10 0.072 2...,date 2006-01-05 1.0 2006-01-10 0.8 2006-...,date 2006-01-05 2.66 2006-01-10 3.86 200...,2006,Alpowa,28.333333,28.333333,179.666667,10.766667,60.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31120,walla_walla,2009-04-16,2009-08-19,date 2009-01-05 79.533333 2009-01-10 76....,date 2009-01-05 2.000000e-01 2009-01-10 ...,date 2009-01-05 4246.60 2009-01-10 12...,date 2009-01-05 1.55 2009-01-10 3.32 200...,date 2009-01-05 62.5 2009-01-10 7.0 2...,date 2009-01-05 7.60 2009-01-10 5.80 2...,date 2009-01-05 9.533333 2009-01-10 2.20...,...,date 2009-01-05 0.121667 2009-01-10 0.22...,date 2009-01-05 0.666667 2009-01-10 0.80...,date 2009-01-05 9.50 2009-01-10 11.82 2...,2009,Alpowa,34.666667,34.666667,168.000000,10.700000,60.900000
31170,walla_walla,2010-03-05,2010-08-19,date 2009-12-01 70.42 2009-12-06 66.76 2...,date 2009-12-01 4.650000e+00 2009-12-06 -...,date 2009-12-01 21315.60 2009-12-06 2135...,date 2009-12-01 2.21 2009-12-06 0.47 200...,date 2009-12-01 332.0 2009-12-06 337.0 2...,date 2009-12-01 8.40 2009-12-06 6.48 2...,date 2009-12-01 14.000 2009-12-06 21.640...,...,date 2009-12-01 0.262 2009-12-06 0.158 2...,date 2009-12-01 0.2 2009-12-06 0.2 2009-...,date 

In [55]:
# save to pickle file
output_filename = raw_dir + "super_df_3months_prior_planting_with_traits.sav"
with open(output_filename, 'wb') as f:
    pickle.dump(merged_super_trait, f)

In [44]:
split_year = 2020

train_df = merged_super_trait[merged_super_trait["year"] < split_year].copy()
test_df  = merged_super_trait[merged_super_trait["year"] >= split_year].copy()

In [45]:
train_df

,location,planting_date,harvest_date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,...,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,year,variety,grain_yield,plant_height,heading_date,protein_content,test_weight
0,almira,2002-04-08,2002-08-12,date 2002-01-05 88.62 2002-01-10 86.42 2...,date 2002-01-05 -1.67 2002-01-10 1.32 2...,date 2002-01-05 0.00 2002-01-10 19....,date 2002-01-05 0.00 2002-01-10 1.02 200...,date 2002-01-05 2.0 2002-01-10 7.0 2...,date 2002-01-05 6.76 2002-01-10 5.86 2...,date 2002-01-05 9.280 2002-01-10 5.840...,...,date 2002-01-05 0.056 2002-01-10 0.084 2...,date 2002-01-05 0.4 2002-01-10 0.4 2002-...,date 2002-01-05 2.10 2002-01-10 2.90 200...,2002,Alpowa,25.666667,25.666667,NaN,11.300000,58.866667
1,almira,2002-04-08,2002-08-12,date 2002-01-05 88.62 2002-01-10 86.42 2...,date 2002-01-05 -1.67 2002-01-10 1.32 2...,date 2002-01-05 0.00 2002-01-10 19....,date 2002-01-05 0.00 2002-01-10 1.02 200...,date 2002-01-05 2.0 2002-01-10 7.0 2...,date 2002-01-05 6.76 2002-01-10 5.86 2...,date 2002-01-05 9.280 2002-01-10 5.840...,...,date 2002-01-05 0.056 2002-01-10 0.084 2...,date 2002-01-05 0.4 2002-01-10 0.4 2002-...,date 2002-01-05 2.10 2002-01-10 2.90 200...,2002,Alturas,25.666667,25.666667,NaN,11.766667,57.433333
2,almira,2002-04-08,2002-08-12,date 2002-01-05 88.62 2002-01-10 86.42 2...,date 2002-01-05 -1.67 2002-01-10 1.32 2...,date 2002-01-05 0.00 2002-01-10 19....,date 2002-01-05 0.00 2002-01-10 1.02 200...,date 2002-01-05 2.0 2002-01-10 7.0 2...,date 2002-01-05 6.76 2002-01-10 5.86 2...,date 2002-01-05 9.280 2002-01-10 5.840...,...,date 2002-01-05 0.056 2002-01-10 0.084 2...,date 2002-01-05 0.4 2002-01-10 0.4 2002-...,date 2002-01-05 2.10 2002-01-10 2.90 200...,2002,BR2306,26.333333,26.333333,NaN,11.733333,58.366667
3,almira,2002-04-08,2002-08-12,date 2002-01-05 88.62 2002-01-10 86.42 2...,date 2002-01-05 -1.67 2002-01-10 1.32 2...,date 2002-01-05 0.00 2002-01-10 19....,date 2002-01-05 0.00 2002-01-10 1.02 200...,date 2002-01-05 2.0 2002-01-10 7.0 2...,date 2002-01-05 6.76 2002-01-10 5.86 2...,date 2002-01-05 9.280 2002-01-10 5.840...,...,date 2002-01-05 0.056 2002-01-10 0.084 2...,date 2002-01-05 0.4 2002-01-10 0.4 2002-...,date 2002-01-05 2.10 2002-01-10 2.90 200...,2002,BR7030,26.000000,26.000000,NaN,12.466667,61.300000
4,almira,2002-04-08,2002-08-12,date 2002-01-05 88.62 2002-01-10 86.42 2...,date 2002-01-05 -1.67 2002-01-10 1.32 2...,date 2002-01-05 0.00 2002-01-10 19....,date 2002-01-05 0.00 2002-01-10 1.02 200...,date 2002-01-05 2.0 2002-01-10 7.0 2...,date 2002-01-05 6.76 2002-01-10 5.86 2...,date 2002-01-05 9.280 2002-01-10 5.840...,...,date 2002-01-05 0.056 2002-01-10 0.084 2...,date 2002-01-05 0.4 2002-01-10 0.4 2002-...,date 2002-01-05 2.10 2002-01-10 2.90 200...,2002,BZ998-447W,30.000000,30.000000,NaN,11.700000,59.566667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31931,weiser,2018-04-05,2018-07-13,date 2018-01-05 69.19 2018-01-10 69.59 2...,date 2018-01-05 -3.12 2018-01-10 1.75 2...,date 2018-01-05 1.65 2018-01-10 31....,date 2018-01-05 0.33 2018-01-10 0.90 201...,date 2018-01-05 2.0 2018-01-10 7.0 2...,date 2018-01-05 9.74 2018-01-10 5.16 2...,date 2018-01-05 12.220 2018-01-10 5.060...,...,date 2018-01-05 0.186 2018-01-10 0.208 2...,date 2018-01-05 0.2 2018-01-10 1.0 2018-...,date 2018-01-05 1.06 2018-01-10 1.52 201...,2018,Ryan,35.320400,NaN,NaN,NaN,58.200000
31932,weiser,2018-04-05,2018-07-13,date 2018-01-05 69.19 2018-01-10 69.59 2...,date 2018-01-05 -3.12 2018-01-10 1.75 2...,date 2018-01-05 1.65 2018-01-10 31....,date 2018-01-05 0.33 2018-01-10 0.90 201...,date 2018-01-05 2.0 2018-01-10 7.0 2...,date 2018-01-05 9.74 2018-01-10 5.16 2...,date 2018-01-05 12.220 2018-01-10 5.060...,...,date 2018-01-05 0.186 2018-01-10 0.208 2...,date 2018-01-05 0.2 2018-01-10 1.0 2018-...,date 2018-01-05 1.06 2018-01-10 1.52 201...,2018,Seahawk,35.990400,NaN,NaN,NaN,60

In [52]:
# check points for location
train_df_check = train_df[train_df["location"] == "aberdeen"]
train_df_check

,location,planting_date,harvest_date,mean_avg_rh_perc,mean_avg_temp_c,sum_cum_gdd_cday,mean_dailyGDD_diff_cday,mean_dap,mean_diurnal_temp_range_c,mean_freezing_dd_cday,...,mean_vpd_kpa,mean_wet_day_frequency_days,mean_wind_speed_ms,year,variety,grain_yield,plant_height,heading_date,protein_content,test_weight
